In [1]:
from math import log, tan, radians, cos, pi, floor, degrees, atan, sinh


def sec(x):
    return(1/cos(x))


def latlon_to_xyz(lat, lon, z):
    tile_count = pow(2, z)
    x = (lon + 180) / 360
    y = (1 - log(tan(radians(lat)) + sec(radians(lat))) / pi) / 2
    return(tile_count*x, tile_count*y)

In [2]:
def bbox_to_xyz(lon_min, lon_max, lat_min, lat_max, z):
    x_min, y_max = latlon_to_xyz(lat_min, lon_min, z)
    x_max, y_min = latlon_to_xyz(lat_max, lon_max, z)
    return(floor(x_min), floor(x_max),
           floor(y_min), floor(y_max))

In [3]:
from geopy.geocoders import Nominatim
from geopy import distance

# create a geocoder instance
geolocator = Nominatim(user_agent='my_app')

# get the coordinates of the city or region
location = geolocator.geocode('Valdemoro, Madrid')

# calculate the size of the square
size = 10  # in kilometers
lat, lon = location.latitude, location.longitude
lat_delta = distance.distance(kilometers=size).destination((lat, lon), bearing=0).latitude - lat
lon_delta = distance.distance(kilometers=size).destination((lat, lon), bearing=90).longitude - lon

# calculate the minimum and maximum latitude and longitude coordinates
lat_min, lat_max = lat - lat_delta, lat + lat_delta
lon_min, lon_max = lon - lon_delta, lon + lon_delta
zoom = 12
# print the results
print(f'Minimum latitude: {lat_min:.6f}')
print(f'Maximum latitude: {lat_max:.6f}')
print(f'Minimum longitude: {lon_min:.6f}')
print(f'Maximum longitude: {lon_max:.6f}')

Minimum latitude: 40.098802
Maximum latitude: 40.278919
Minimum longitude: -3.788859
Maximum longitude: -3.554001


In [4]:
import folium

# Calculate the center of the bounding box
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

# Create a folium Map instance centered on the bounding box, with the desired zoom level
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, scrollWheelZoom=False)

# Add a rectangle to the map to represent the bounding box
folium.Rectangle(
    bounds=[[lat_min, lon_min], [lat_max, lon_max]],
    fill=False,
    color="blue",
).add_to(m)

# Display the map in the Jupyter Notebook
m

In [5]:
legend = {
    "items": [
        {"name": "Water", "color": "#419BDF", "id": 0},
        {"name": "Trees", "color": "#397D49", "id": 1},
        {"name": "Grass", "color": "#88B053", "id": 2},
        {"name": "Flooded Vegetation", "color": "#7A87C6", "id": 3},
        {"name": "Crops", "color": "#E49635", "id": 4},
        {"name": "Shrub and Scrub", "color": "#DFC35A", "id": 5},
        {"name": "Built", "color": "#C4281B", "id": 6},
        {"name": "Bare", "color": "#A59B8F", "id": 7},
        {"name": "Snow and Ice", "color": "#B39FE1", "id": 8},
    ]
}

In [6]:
import ee
import folium
from folium import plugins

ee.Initialize()

# Define the area of interest
aoi = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

# Load the image collection
collection = ee.ImageCollection("projects/wri-datalab/dynamicworld/rw/rgb/DW_202207")

# Filter the collection to only include images that intersect the area of interest
filtered_collection = collection.filterBounds(aoi)

# Create a mosaic from the filtered collection
land_cover = filtered_collection.mosaic()

# Clip the dataset to the area of interest
land_cover = land_cover.clip(aoi)

# Function to convert hex color to a list of RGB values
def hex_to_rgb(hex_color):
    return [int(hex_color[i:i+2], 16) for i in (1, 3, 5)]

# Calculate the area for each class using color values
stats = {}

for class_info in legend["items"]:
    class_id = class_info['id']
    class_color = ee.Image.constant(hex_to_rgb(class_info['color'])).toFloat()

    mask = land_cover.eq(class_color).reduce(ee.Reducer.allNonZero())

    class_area = mask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10,
        maxPixels=1e9
    )

    stats[class_id] = {
        "name": class_info["name"],
        "area": class_area.getInfo(),
    }

print(stats)

# Function to add Earth Engine layers to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
my_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=12, scrollWheelZoom=False)

# Add the land cover layer to the map
vis_params = {
    'bands': ['vis-red', 'vis-green', 'vis-blue'],
    'min': 0,
    'max': 255,
    'gamma': 1.0
}
my_map.add_ee_layer(land_cover, vis_params, 'Land Cover')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)


{0: {'name': 'Water', 'area': {'all': 12096.894117647058}}, 1: {'name': 'Trees', 'area': {'all': 156525.03529411764}}, 2: {'name': 'Grass', 'area': {'all': 9561}}, 3: {'name': 'Flooded Vegetation', 'area': {'all': 2174}}, 4: {'name': 'Crops', 'area': {'all': 2451052.619607867}}, 5: {'name': 'Shrub and Scrub', 'area': {'all': 558483.6549019614}}, 6: {'name': 'Built', 'area': {'all': 830611.9019607825}}, 7: {'name': 'Bare', 'area': {'all': 1221206.5098039226}}, 8: {'name': 'Snow and Ice', 'area': {'all': 10}}}


In [7]:

import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import json
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

In [ ]:
from IPython.display import Markdown
response = chat(
    [
        SystemMessage(content="You are a decision-maker responsible for the development and management of a given area. You have been provided with land cover statistics from the WRI Dynamic World dataset for your area, which includes information about various land cover types, such as water, trees, grass, crops, built-up areas, and more."),
        AIMessage(content=json.dumps(stats)),
        HumanMessage(content="Analyze this data provided and give extensive insights about the characteristics of the terrain, possible opportunities, environmental risks and challenges, advice on infrastructure development, and any other relevant information that can help in formulating sustainable policies and strategies for the provided data area. Also, show the statistics provided in a comprehensive format")
    ]
)

aimessage_content = response.content

display(Markdown(aimessage_content))